In [42]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [43]:
# Import OpenAI chat model
from langchain.chat_models import ChatOpenAI
import os

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Create the OpenAI chat model
chat = ChatOpenAI(
    model="gpt-4",  # Use "gpt-3.5-turbo" if you want a cheaper alternative
    temperature=0.2
)

Tool 1: web search

In [55]:
from langchain_community.tools import TavilySearchResults, DuckDuckGoSearchResults
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool

os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

# internet_search = TavilySearchResults(
#     max_results=10,
#     include_answer=True,
#     include_raw_content=True,
#     include_images=False
#     )
# internet_search = DuckDuckGoSearchResults(max_results=10)

# Initialize the SerpAPI search tool
serp_api = SerpAPIWrapper()

# Wrap it as a Tool to add name & description
internet_search = Tool(
    name="internet_search",
    description="Returns a list of relevant document snippets for a textual query retrieved from the internet.",
    func=serp_api.run  # SerpAPIWrapper does not support direct execution, so use `.run`
)

# internet_search.name = "internet_search"
# internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


In [36]:
# internet_search.invoke({'query': 'Create a plot of the monthly average of gold price in swiss franc by gramm for the last 10 years'})

Tool 2: plot

In [56]:
import matplotlib

from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
# class ToolInput(BaseModel):
#     code: str = Field(description="Python code to execute.")
# repl_tool.args_schema = ToolInput

Agent

In [57]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate


# Create the prompt template
# prompt = ChatPromptTemplate.from_template("{input}")
# Create the prompt with agent_scratchpad
prompt = ChatPromptTemplate.from_template(
    "{input}\n\nPrevious Actions:\n{agent_scratchpad}"
)

# Initialize the ReAct agent
agent = create_openai_tools_agent(
    tools=[internet_search, repl_tool],
    llm=chat,
    prompt=prompt,
)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=[internet_search, repl_tool], verbose=True)


In [58]:
try:
    result = agent_executor.invoke({
        "input":"Plot the gold price history in us dollar by once for the last week"
    })
except AttributeError as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...

Invoking: `internet_search` with `gold price history in us dollar for the last week data`


['(XAUUSD) Prices logged 3:00 Mountain time daily* ; Yesterday · 2,760.83 · 17.40 ; Last Week · 2,768.16 · 10.07 ; Last Month · 2,624.49 · 153.74.', 'Gold Price Performance USD ; 30 Days, +109.71, +4.18% ; 6 Months, +324.73, +13.48% ; 1 Year, +697.61, +34.25% ; 5 Year, +1,150.70, +72.67%.', 'All Data gold Price History in US Dollars per Ounce. Receive Gold and Silver Price Updates via Email.', 'Live gold prices in US Dollars, British Pounds, Euros, Japanese Yen ... Gold Price Last Week. Current Price, $89,302.24. Week High, $89,564.99. Week Low ...', 'Gold Futures Historical Data ; Highest: 2,822.10 ; Change %:. 6.61 ; Average: 2,718.38 ; Difference: 213.70 ; Lowest: 2,608.40.', 'Discover historical prices for GC=F stock on Yahoo Finance. View daily, weekly or monthly format back to when Gold Feb 25 stock was issued.', 'View the all-time historical gold p